In [ ]:
import numpy as np
import pandas as pd
import pooch
from scipy.optimize import root

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
bc_1850 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'1850'].values[0]
oc_1850 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'1850'].values[0]
so2_1850 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'1850'].values[0]

bc_2014 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'2014'].values[0]
oc_2014 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'2014'].values[0]
so2_2014 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'2014'].values[0]

In [ ]:
bc_1850

In [ ]:
bc_2014

In [ ]:
# Fiona reports IRF as ERFari and RA as ERFaci, where IRF is calculated using the Ghan 2012 method (2013 I think).
# This is same as Thornhill et al. 2021.
target_erfari_so2 = -0.49
target_erfari_bc = +0.38
target_erfari_oc = -0.14
target_erfaci_so2 = -0.88
target_erfaci_bc = -0.01
target_erfaci_oc = -0.08
target_erfaci = -0.84

# O'Connor et al. 2021: https://acp.copernicus.org/articles/21/1211/2021/acp-21-1211-2021.html
# Individual forcing experiments:
# Sulfate ERF = -1.37
# BC ERF      = +0.37
# OC ERF      = -0.22
# total       = -1.22

# All species varied together = -1.09

# Thornhill et al. 2021 https://acp.copernicus.org/articles/21/853/2021/acp-21-853-2021.html

# Fiona sweeps up ERFcs,af into the RA term, which is dominated by ERFaci. Ghan attributes this to surface albedo,
# Thornhill says yes albedo, plus other tropospheric adjustments.

In [ ]:
def erf_rootfinder(x):
    alpha_so2, alpha_bc, alpha_oc, s_so2, s_bc, s_oc, beta = x
    erfari_so2 = (so2_2014 - so2_1850) * alpha_so2
    erfari_bc = (bc_2014 - bc_1850) * alpha_bc
    erfari_oc = (oc_2014 - oc_1850) * alpha_oc
    erfaci_so2 = beta * (np.log(1 + s_so2 * so2_2014) - np.log(1 + s_so2 * so2_1850))
    erfaci_bc = beta * (np.log(1 + s_bc * bc_2014) - np.log(1 + s_bc * bc_1850))
    erfaci_oc = beta * (np.log(1 + s_oc * oc_2014) - np.log(1 + s_oc * oc_1850))
    erfaci = beta * (np.log(1 + s_so2 * so2_2014 + s_bc * bc_2014 + s_oc * oc_2014) - np.log(1 + s_so2 * so2_1850 + s_bc * bc_1850 + s_oc * oc_1850))
    return np.array(
        [
            erfari_so2 - target_erfari_so2, 
            erfari_bc - target_erfari_bc, 
            erfari_oc - target_erfari_oc, 
            erfaci_so2 - target_erfaci_so2, 
            erfaci_bc - target_erfaci_bc, 
            erfaci_oc - target_erfaci_oc, 
            erfaci - target_erfaci
        ]
    )

In [ ]:
root(
    erf_rootfinder, 
    np.array(
        [
            -0.0043,
            0.05158,
            -0.0052,
            0.037031928,
            0.001,
            0.016231426,
            -0.735769463,
        ],
    ),
    method='lm',
    options={'maxiter': 500000}
)